In [1]:
import gensim
import logging
import json

'''
要自己訓練的話用這個，setence是一個list，裡面每個element則是由切好的詞構成的list


例如：
sentence = [['今天','實在','很熱'],['感覺','路面','可以','煎蛋']]

模型參數看官方文件說明：https://radimrehurek.com/gensim/models/word2vec.html
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = gensim.models.Word2Vec(sentence, size=200, workers=2, min_count=1, sg=1)
'''
model = gensim.models.Word2Vec.load('./mymodelSG')

In [2]:
# 相似度
for ele in model.most_similar(u'滷肉飯', topn=10):
    print ele[0], ele[1]

魯肉飯 0.893000423908
肉燥飯 0.810384392738
控肉飯 0.757050454617
爌肉飯 0.754742383957
爌肉 0.746101021767
焢肉 0.742386460304
滷肉 0.740453720093
焢肉飯 0.7402151227
肉燥 0.733509778976
控肉 0.723708808422


In [3]:
# KMean前處理
# 詞清單
wordlist = model.index2word
# 詞向量
wordVector = model.syn0
wordVectorShape=wordVector.shape[0]
print  wordVectorShape
num_clusters = wordVectorShape/ 5
print num_clusters

398925
79785


In [6]:

print  type(wordlist)
print len(wordlist)
print json.dumps(wordlist[1:20],ensure_ascii=False).encode('utf-8')
print '------------------------------------------------------------------------------------------------------'
print  type(wordVector)
print len(wordVector)
print(wordVector[0])
print len(wordVector[0])

print(wordVector.ndim)
print(wordVector.shape)
print(wordVector.dtype)

<type 'list'>
398925
["是", "了", "有", "我", "吃", "就", "在", "很", "也", "來", "這", "但", "和", "都", "可以", "到", "我們", "就是", "元"]
------------------------------------------------------------------------------------------------------
<type 'numpy.ndarray'>
398925
[ 0.28682771 -0.07586922  0.1578736   0.11840189  0.16901097 -0.09892504
  0.03410177  0.00984159  0.29855445 -0.12518986  0.09773253  0.1572655
 -0.13239793  0.04892337 -0.13496728  0.4081839   0.3338187  -0.03503511
  0.11309864 -0.17892414 -0.08576016  0.25438815  0.15055723 -0.05878566
  0.24155891  0.26643324  0.14156783  0.30228481  0.12343887  0.35488629
 -0.06419866 -0.09948944  0.14322282  0.34893551  0.25273603  0.09931388
  0.18456919  0.10962344 -0.12785533  0.07407416 -0.00332142  0.13633065
  0.1573704  -0.05271621  0.0078309  -0.2508778  -0.16921134 -0.08169635
  0.13334352 -0.29259396  0.19113475  0.00621219  0.16497807 -0.07298919
  0.10587206 -0.33191103 -0.07477511  0.57821494  0.08147391  0.26024202
  0.23402673 -0.14

In [15]:
'''
[Spectral Clustering 前處理]

import numpy as np

全部詞算相似度

smatrix = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(model.syn0.T))
nsarray = np.array(smatrix) <=== 會當機

若已經把詞的範圍縮減，則可用下面這個方式
每個詞的向量長度是n，訓練時我是將n設為200
接著把詞向量放入list

wordVectorList = [wordvector1,wordvector2,...,...]
nsmatrix = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(np.array(wordVectorList).T))

算出來的相似度因為是浮點數，會有>1或<-1的情況，要再修正
smatrix = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(model.syn0.T))
nsarray = np.array(smatrix) smatrix = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(model.syn0.T))
nsarray = np.array(smatrix) 
nsarray = np.array(nsmatrix)
nsarray[nsarray>1] = 1.0
nsarray[nsarray<-1] = -1.0

另外可以把餘弦相似度轉換為弧度，再轉換為介於0~1之間的相似度

angularD = np.arccos(nsarray)/math.pi
sim = 1 - angularD

'''

'\n[Spectral Clustering \xe5\x89\x8d\xe8\x99\x95\xe7\x90\x86]\n\nimport numpy as np\n\n\xe5\x85\xa8\xe9\x83\xa8\xe8\xa9\x9e\xe7\xae\x97\xe7\x9b\xb8\xe4\xbc\xbc\xe5\xba\xa6\n\nsmatrix = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(model.syn0.T))\nnsarray = np.array(smatrix) <=== \xe6\x9c\x83\xe7\x95\xb6\xe6\xa9\x9f\n\n\xe8\x8b\xa5\xe5\xb7\xb2\xe7\xb6\x93\xe6\x8a\x8a\xe8\xa9\x9e\xe7\x9a\x84\xe7\xaf\x84\xe5\x9c\x8d\xe7\xb8\xae\xe6\xb8\x9b\xef\xbc\x8c\xe5\x89\x87\xe5\x8f\xaf\xe7\x94\xa8\xe4\xb8\x8b\xe9\x9d\xa2\xe9\x80\x99\xe5\x80\x8b\xe6\x96\xb9\xe5\xbc\x8f\n\xe6\xaf\x8f\xe5\x80\x8b\xe8\xa9\x9e\xe7\x9a\x84\xe5\x90\x91\xe9\x87\x8f\xe9\x95\xb7\xe5\xba\xa6\xe6\x98\xafn\xef\xbc\x8c\xe8\xa8\x93\xe7\xb7\xb4\xe6\x99\x82\xe6\x88\x91\xe6\x98\xaf\xe5\xb0\x87n\xe8\xa8\xad\xe7\x82\xba200\n\xe6\x8e\xa5\xe8\x91\x97\xe6\x8a\x8a\xe8\xa9\x9e\xe5\x90\x91\xe9\x87\x8f\xe6\x94\xbe\xe5\x85\xa5list\n\nwordVectorList = [wordvector1,wordvector2,...,...]\nnsmatrix = gensim.similarities.MatrixSi

KeyboardInterrupt: 

In [12]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

# Set "k" (num_clusters) to be 1/5th of the vocabulary size, or an
# average of 5 words per cluster


word_vectors = model.syn0
num_clusters =15

# Initalize a k-means object and use it to extract centroids
kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

# Get the end time and print how long the process took
end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

# Create a Word / Index dictionary, mapping each vocabulary word to
# a cluster number                                                                                            
word_centroid_map = dict(zip( model.index2word, idx ))

# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words

Time taken for K Means clustering:  1711.21886206 seconds.

Cluster 0
Traceback (most recent call last):
  File "/home/dondon/App/Anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1120, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/dondon/App/Anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 301, in wrapped
    return f(*args, **kwargs)
  File "/home/dondon/App/Anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 346, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/dondon/App/Anaconda2/lib/python2.7/inspect.py", line 1049, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))
  File "/home/dondon/App/Anaconda2/lib/python2.7/inspect.py", line 1009, in getframeinfo
    filename = getsourcefile(frame) or getfile(frame)
  File "/home/dondon/App/Anaconda2/lib/python2.7/inspect.py"

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



IndexError: string index out of range